In [8]:
import pandas as pd

# Подключаю датасет
url = "https://github.com/YBI-Foundation/Dataset/raw/main/NSE%20Gold%20CPI%20.xlsx" #Да, там эксель
data = pd.read_excel(url, engine='openpyxl') #Понадобиться pip install openpyxl

# Проверяем, как выглядят данные
print(data.head())

        Date           NSE    Gold    CPI
0 2021-07-01  15763.049805  1812.6  162.5
1 2021-06-01  15721.500000  1770.8  161.3
2 2021-05-01  15582.799805  1902.5  160.4
3 2021-04-01  14631.099609  1767.3  157.8
4 2021-03-01  14690.700195  1713.8  156.8


In [9]:
# Преобразуем столбец с датой в формат datetime
data['Date'] = pd.to_datetime(data['Date'])

# 1. Функция для apply / для считавание разницы между максимальным и минимальным значениями в группе
def range_func(x):
    return x.max() - x.min()

# Применяем функцию к сгруппированным данным по году с помощью apply
result_apply = data.groupby(data['Date'].dt.year)['Gold'].apply(range_func)
print("Результат функции apply:\n", result_apply)

Результат функции apply:
 Date
2017    122.0
2018    138.7
2019    327.6
2020    502.0
2021    188.7
Name: Gold, dtype: float64


In [10]:
# 2. Функция для transform / для нормировании данных каждого месяца к их максимальному значению
def normalize(x):
    return x / x.max()

# Применяем функцию transform к данным, сгруппированным по месяцу
data['Normalized_Gold'] = data.groupby(data['Date'].dt.month)['Gold'].transform(normalize)
print("\nРезультат функции transform:\n", data[['Date', 'Gold', 'Normalized_Gold']].head())


Результат функции transform:
         Date    Gold  Normalized_Gold
0 2021-07-01  1812.6              1.0
1 2021-06-01  1770.8              1.0
2 2021-05-01  1902.5              1.0
3 2021-04-01  1767.3              1.0
4 2021-03-01  1713.8              1.0


In [11]:
# 3. Функция для map / округление значений столбца Gold до целого числа
data['Rounded_Gold'] = data['Gold'].map(lambda x: round(x))
print("\nРезультат функции map:\n", data[['Date', 'Gold', 'Rounded_Gold']].head())


Результат функции map:
         Date    Gold  Rounded_Gold
0 2021-07-01  1812.6          1813
1 2021-06-01  1770.8          1771
2 2021-05-01  1902.5          1902
3 2021-04-01  1767.3          1767
4 2021-03-01  1713.8          1714


In [12]:
# 4. Ресемплирование данных по месяцам, вычисляем среднее значение по каждому месяцу
monthly_data = data.resample('ME', on='Date').mean()
print("\nРесемплированные данные по месяцам:\n", monthly_data[['Gold', 'Normalized_Gold']].head())


Ресемплированные данные по месяцам:
               Gold  Normalized_Gold
Date                               
2017-06-30  1271.5         0.718037
2017-07-31  1170.8         0.645923
2017-08-31  1150.0         0.585898
2017-09-30  1252.6         0.636613
2017-10-31  1247.3         0.660821


### Итак, делаю пометки для себя с привязкой к трактовке данных:

- Функция с apply: вычисление диапазона (разницы между максимальным и минимальным значением) для определенных групп (например, по годам) помогает анализировать волатильность цены золота в каждый год. Такой подход полезен для выявления рисков и тенденций.

- Функция с transform: нормализация данных по месяцам позволяет сравнивать значения относительно максимума месяца. Это часто используется в сезонных и временных рядах для выявления отклонений от обычного поведения.

- Функция с map: округление данных позволяет сделать значения более читабельными, что полезно для отчётов и визуализаций.

- Ресемплирование с resample: агрегирование данных в более крупные интервалы (например, месячные средние значения) полезно для выявления долгосрочных трендов и сглаживания шума в данных.

Можно юзать для анализа рынка акций, товарных рынков и других временных рядов, чтобы принимать более обоснованные решения.